In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import os
import statsmodels.api as sm
#from graphviz import Source
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
#from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import mapi
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [3]:
#Load Raw Data
file_path = 'C:\\Users\\rafma\\Python\\46889 Capstone\\GenericWellData.xlsx'
#Created data frame of loaded data
df = pd.read_excel(file_path)
#Define Well Index
index = df['Well Index']

In [4]:
#Exclude predictor variables for which missing data will not be imputed. Data frame limited to well depth data and material input data for well production
x = df.drop(['FractureStages', 'AvgStageSpacing_FT', 'Well Index', 'InitialProductionDate', 'DrillingStartDate','InitialProductionYear', 'DrillingCompletionDate', 'YearOfDrilling', 'InitialProductionMonth', 'ProductionMonthsCount', 'First36MonthWater_BBL', 'First3MonthGas_MCF', 'First3MonthGas_MCFPer1000FT', 'First3MonthProd_MCFE', 'First3MonthProd_MCFEPer1000FT', 'First3MonthOil_BBL', 'First3MonthOil_BBLPer1000FT', 'First3MonthWater_BBL', 'First6MonthProd_BOE', 'First6MonthProd_BOEPer1000FT', 'First6MonthGas_MCF', 'First6MonthGas_MCFPer1000FT', 'First6MonthProd_MCFE', 'First6MonthProd_MCFEPer1000FT', 'First6MonthOil_BBL', 'First6MonthOil_BBLPer1000FT', 'First6MonthWater_BBL', 'First9MonthProd_BOE', 'First9MonthProd_BOEPer1000FT', 'First9MonthGas_MCF', 'First9MonthGas_MCFPer1000FT', 'First9MonthProd_MCFE', 'First9MonthProd_MCFEPer1000FT', 'First9MonthOil_BBL', 'First9MonthOil_BBLPer1000FT', 'First9MonthWater_BBL', 'First12MonthProd_BOE', 'First12MonthProd_BOEPer1000FT', 'First12MonthGas_MCF', 'First12MonthGas_MCFPer1000FT', 'First12MonthProd_MCFE', 'First12MonthProd_MCFEPer1000FT', 'First12MonthOil_BBL', 'First12MonthOil_BBLPer1000FT', 'First12MonthWater_BBL', 'First36MonthProd_BOE', 'First36MonthProd_BOEPer1000FT', 'First36MonthGas_MCFPer1000FT', 'First36MonthProd_MCFE', 'First36MonthProd_MCFEPer1000FT', 'First36MonthOil_BBLPer1000FT', 'First36MonthWaterProductionBBLPer1000Ft', 'PeakProd_BOE', 'PeakProd_BOEPer1000FT', 'PeakGas_MCF', 'PeakGas_MCFPer1000FT', 'PeakProd_MCFE', 'PeakProd_MCFEPer1000FT', 'PeakOil_BBL', 'PeakOil_BBLPer1000FT', 'PeakWater_BBL', 'CumProd_BOE', 'CumProd_BOEPer1000FT', 'CumGas_MCF', 'CumGas_MCFPer1000FT', 'CumProd_MCFE', 'CumProd_MCFEPer1000FT', 'CumOil_BBL', 'CumOil_BBLPer1000FT', 'CumWater_BBL', 'First3MonthProd_BOEPer1000FT'], axis = 1)

In [5]:
#Generate dummy variables for categorical OilTest Field
x = pd.get_dummies(x, columns=['OilTest_Method'])
for col in x.columns:
    if x[col].dtype == 'bool':
        x[col] = x[col].astype(int)

In [6]:
for col in x.columns:
    if x[col].dtype == 'object':  # Check if the column contains string values
        # Remove commas and convert to numeric, setting errors to 'coerce' to handle non-numeric strings
        x[col] = pd.to_numeric(x[col].str.replace(',', ''), errors='coerce')

In [7]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TrueVerticalDepth_FT        16705 non-null  float64
 1   MeasuredDepth_FT            16660 non-null  float64
 2   UpperPerforation_FT         16573 non-null  float64
 3   LowerPerforation_FT         16573 non-null  float64
 4   PerforationInterval_FT      16659 non-null  float64
 5   LateralLength_FT            16705 non-null  float64
 6   ProppantLoad_LBSPerGAL      15188 non-null  float64
 7   ProppantIntensity_LBSPerFT  15324 non-null  float64
 8   TotalProppant_LBS           15368 non-null  float64
 9   TotalWaterPumped_GAL        15682 non-null  float64
 10  WaterIntensity_GALPerFT     15662 non-null  float64
 11  TotalFluidPumped_BBL        15662 non-null  float64
 12  FluidIntensity_BBLPerFT     15642 non-null  float64
 13  AcidVolume_BBL              134

In [8]:
#ONLY USED FOR K DETERMINATION
#Train Test Split for XGBOOST KNN
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=99)

In [9]:
#ONLY USED FOR K DETERMINATION - Run this ONLY if you want to see the process for determining optimal k value
# Dictionary to store the scores for different k values
# scores = {}

# # Iterate over multiple k values
# for k in range(1, 15):  # Example: trying k values from 1 to 5
#     #Create kNN Imputer
#     imputer = KNNImputer(n_neighbors=k)
    
#     # Fit the imputer and transform the training data
#     x_train_imputed = imputer.fit_transform(x_train)
#     x_test_imputed = imputer.transform(x_test)
    
#     # Fit a simple model (RF OR XGBOOST) to the imputed data
#     #model = RandomForestRegressor(random_state=99) if RF model desired for optimal k determination
#     model = XGBRegressor(learning_rate=0.075, n_estimators= 400, max_depth = 7, random_state=99, alpha = 0.5)
#     model.fit(x_train_imputed, y_train)
    
#     # Predict on the test set and evaluate the performance
#     y_pred = model.predict(x_test_imputed)
#     #accuracy = accuracy_score(y_test, y_pred)
    
#     #Evaluate the performance using R-squared or or other performance metric
#     r2 = r2_score(y_test, y_pred)
#     #mae = mean_absolute_error(y_test, y_pred)
#     #mse = mean_squared_error(y_test, y_pred)
#    # rmse = np.sqrt(mse)
    
#     # Store the score
#     scores[k] = r2
#     #scores[k] = accuracy
    
# # Find the best k value
# best_k = max(scores, key=scores.get)
# best_score = scores[best_k]

# print(f"Best k: {best_k} with R^2: {best_score}")
# print(scores)

In [10]:
#Create Missing Data Iterative Imputer functional with optimal k, k= 7
imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=7))

In [11]:
#Run KNN iterative imputer on the 
imputed_data = imputer.fit_transform(x)
imputed_x = pd.DataFrame(imputed_data, columns=x.columns)

C:\Users\rafma\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [12]:
imputed_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TrueVerticalDepth_FT        16707 non-null  float64
 1   MeasuredDepth_FT            16707 non-null  float64
 2   UpperPerforation_FT         16707 non-null  float64
 3   LowerPerforation_FT         16707 non-null  float64
 4   PerforationInterval_FT      16707 non-null  float64
 5   LateralLength_FT            16707 non-null  float64
 6   ProppantLoad_LBSPerGAL      16707 non-null  float64
 7   ProppantIntensity_LBSPerFT  16707 non-null  float64
 8   TotalProppant_LBS           16707 non-null  float64
 9   TotalWaterPumped_GAL        16707 non-null  float64
 10  WaterIntensity_GALPerFT     16707 non-null  float64
 11  TotalFluidPumped_BBL        16707 non-null  float64
 12  FluidIntensity_BBLPerFT     16707 non-null  float64
 13  AcidVolume_BBL              167

In [13]:
#Add back well index to the fields with missing values estimated for re-creating original data with missing values of select comlumns populated
imputed_x.insert(0, "Well Index", index)

In [14]:
imputed_x.head(5)

,Well Index,TrueVerticalDepth_FT,MeasuredDepth_FT,UpperPerforation_FT,LowerPerforation_FT,PerforationInterval_FT,LateralLength_FT,ProppantLoad_LBSPerGAL,ProppantIntensity_LBSPerFT,TotalProppant_LBS,TotalWaterPumped_GAL,WaterIntensity_GALPerFT,TotalFluidPumped_BBL,FluidIntensity_BBLPerFT,AcidVolume_BBL,DrillingDuration_DAYS,OilTest_Method_FLOWING,OilTest_Method_GAS LIFT,OilTest_Method_PUMPING
0,1,11890.0,15995.0,11878.0,15811.0,3933.0,3718.0,4.445714,1802.428571,1.429378e+07,404320.0,103.000000,9627.000000,2.000000,1216.428571,13.285714,0.0,0.0,1.0
1,2,9141.0,13493.0,9470.0,13400.0,3930.0,4076.0,0.770000,672.000000,2.641750e+06,3412481.0,868.000000,81250.000000,21.000000,555.142857,7.428571,0.0,0.0,1.0
2,3,9688.0,13824.0,10082.0,13739.0,3657.0,3943.0,0.990000,1094.000000,4.000020e+06,4051150.0,1108.000000,96456.000000,26.000000,708.857143,6.142857,0.0,0.0,1.0
3,4,9187.0,13542.0,9451.0,13436.0,3985.0,4145.0,0.470000,560.000000,2.233000e+06,4768999.0,1197.000000,113548.000000,28.000000,675.428571,5.142857,0.0,1.0,0.0
4,5,7935.0,12859.0,8478.0,12690.0,4212.0,4564.0,0.841429,1195.000000,5.033010e+06,6029816.0,1420.714286,145108.285714,34.142857,716.857143,7.714286,0.0,0.0,1.0


In [15]:
#GenericWellDataPrepped = pd.merge (imputed_x, on='Key', how, left)

In [16]:
#Run this only if you want to create file of columns for which missing data was estimated with KNN Iterative Imputer
#imputed_x.to_excel('x_data_cleaned_imputed.xlsx', index=False)

In [17]:
#Identify columns dropped for the KNN missing data imputation process
dropped_col = ['Well Index', 'FractureStages', 'AvgStageSpacing_FT', 'InitialProductionDate', 'DrillingStartDate','InitialProductionYear', 'DrillingCompletionDate', 'YearOfDrilling', 'InitialProductionMonth', 'ProductionMonthsCount', 'First36MonthWater_BBL', 'First3MonthGas_MCF', 'First3MonthGas_MCFPer1000FT', 'First3MonthProd_MCFE', 'First3MonthProd_MCFEPer1000FT', 'First3MonthOil_BBL', 'First3MonthOil_BBLPer1000FT', 'First3MonthWater_BBL', 'First6MonthProd_BOE', 'First6MonthProd_BOEPer1000FT', 'First6MonthGas_MCF', 'First6MonthGas_MCFPer1000FT', 'First6MonthProd_MCFE', 'First6MonthProd_MCFEPer1000FT', 'First6MonthOil_BBL', 'First6MonthOil_BBLPer1000FT', 'First6MonthWater_BBL', 'First9MonthProd_BOE', 'First9MonthProd_BOEPer1000FT', 'First9MonthGas_MCF', 'First9MonthGas_MCFPer1000FT', 'First9MonthProd_MCFE', 'First9MonthProd_MCFEPer1000FT', 'First9MonthOil_BBL', 'First9MonthOil_BBLPer1000FT', 'First9MonthWater_BBL', 'First12MonthProd_BOE', 'First12MonthProd_BOEPer1000FT', 'First12MonthGas_MCF', 'First12MonthGas_MCFPer1000FT', 'First12MonthProd_MCFE', 'First12MonthProd_MCFEPer1000FT', 'First12MonthOil_BBL', 'First12MonthOil_BBLPer1000FT', 'First12MonthWater_BBL', 'First36MonthProd_BOE', 'First36MonthProd_BOEPer1000FT', 'First36MonthGas_MCFPer1000FT', 'First36MonthProd_MCFE', 'First36MonthProd_MCFEPer1000FT', 'First36MonthOil_BBLPer1000FT', 'First36MonthWaterProductionBBLPer1000Ft', 'PeakProd_BOE', 'PeakProd_BOEPer1000FT', 'PeakGas_MCF', 'PeakGas_MCFPer1000FT', 'PeakProd_MCFE', 'PeakProd_MCFEPer1000FT', 'PeakOil_BBL', 'PeakOil_BBLPer1000FT', 'PeakWater_BBL', 'CumProd_BOE', 'CumProd_BOEPer1000FT', 'CumGas_MCF', 'CumGas_MCFPer1000FT', 'CumProd_MCFE', 'CumProd_MCFEPer1000FT', 'CumOil_BBL', 'CumOil_BBLPer1000FT', 'CumWater_BBL', 'First3MonthProd_BOEPer1000FT']

In [18]:
#Identify columns dropped for the KNN missing data imputation process
dropped = df[dropped_col]

In [19]:
dropped.head(5)

,Well Index,FractureStages,AvgStageSpacing_FT,InitialProductionDate,DrillingStartDate,InitialProductionYear,DrillingCompletionDate,YearOfDrilling,InitialProductionMonth,ProductionMonthsCount,...,CumProd_BOE,CumProd_BOEPer1000FT,CumGas_MCF,CumGas_MCFPer1000FT,CumProd_MCFE,CumProd_MCFEPer1000FT,CumOil_BBL,CumOil_BBLPer1000FT,CumWater_BBL,First3MonthProd_BOEPer1000FT
0,1,NaN,NaN,2011-02-01,2006-03-04,2011,2006-05-27,2011,2011 / 02,154,...,153536,39038.0,856573,217791.0,921216,234227.0,10774,2739.0,2182,244.0
1,2,NaN,NaN,2011-02-01,2010-11-19,2011,2011-02-05,2011,2011 / 02,153,...,170139,43292.0,491924,125171.0,1020832,259754.0,88151,22430.0,141001,4894.0
2,3,NaN,NaN,2011-03-01,2010-09-13,2011,2011-03-02,2011,2011 / 03,153,...,99049,27085.0,102078,27913.0,594295,162509.0,82036,22433.0,175835,868.0
3,4,NaN,NaN,2011-03-01,2011-01-04,2011,2011-03-16,2011,2011 / 03,152,...,195873,49153.0,337319,84647.0,1175237,294915.0,139653,35045.0,40064,6591.0
4,5,15.0,304.0,2011-04-01,2011-01-17,2011,2011-04-04,2011,2011 / 04,152,...,137857,32730.0,200840,47683.0,827145,196378.0,104384,24783.0,310369,2852.0


In [20]:
#Add back all dropped fields from original well data set to create complete data set with missing values estimated for selected columns
GenericWellDataPrepped = pd.merge (imputed_x, dropped, on='Well Index', how='left')

In [21]:
#This is the final data prepped for creating the training & Test Sets for the Model
GenericWellDataPrepped.head(10)

,Well Index,TrueVerticalDepth_FT,MeasuredDepth_FT,UpperPerforation_FT,LowerPerforation_FT,PerforationInterval_FT,LateralLength_FT,ProppantLoad_LBSPerGAL,ProppantIntensity_LBSPerFT,TotalProppant_LBS,...,CumProd_BOE,CumProd_BOEPer1000FT,CumGas_MCF,CumGas_MCFPer1000FT,CumProd_MCFE,CumProd_MCFEPer1000FT,CumOil_BBL,CumOil_BBLPer1000FT,CumWater_BBL,First3MonthProd_BOEPer1000FT
0,1,11890.0,15995.0,11878.0,15811.0,3933.0,3718.0,4.445714,1802.428571,1.429378e+07,...,153536,39038.0,856573,217791.0,921216,234227.0,10774,2739.0,2182,244.0
1,2,9141.0,13493.0,9470.0,13400.0,3930.0,4076.0,0.770000,672.000000,2.641750e+06,...,170139,43292.0,491924,125171.0,1020832,259754.0,88151,22430.0,141001,4894.0
2,3,9688.0,13824.0,10082.0,13739.0,3657.0,3943.0,0.990000,1094.000000,4.000020e+06,...,99049,27085.0,102078,27913.0,594295,162509.0,82036,22433.0,175835,868.0
3,4,9187.0,13542.0,9451.0,13436.0,3985.0,4145.0,0.470000,560.000000,2.233000e+06,...,195873,49153.0,337319,84647.0,1175237,294915.0,139653,35045.0,40064,6591.0
4,5,7935.0,12859.0,8478.0,12690.0,4212.0,4564.0,0.841429,1195.000000,5.033010e+06,...,137857,32730.0,200840,47683.0,827145,196378.0,104384,24783.0,310369,2852.0
5,6,9048.0,13056.0,9340.0,12935.0,3595.0,3815.0,0.430000,452.000000,1.626000e+06,...,120312,33466.0,237741,66131.0,721870,200798.0,80688,22445.0,64662,3775.0
6,7,10472.0,14522.0,11038.0,14467.0,3429.0,3779.0,0.840000,1215.000000,4.166280e+06,...,116709,34036.0,194882,56833.0,700257,204216.0,84229,24564.0,141109,2004.0
7,8,9285.0,13464.0,9437.0,13314.0,3877.0,4004.0,0.470000,392.000000,1.518162e+06,...,90656,23383.0,248095,63991.0,543937,140298.0,49307,12718.0,46884,2306.0
8,9,9315.0,13624.0,9707.0,13448.0,3741.0,4108.0,0.560000,583.000000,2.182000e+06,...,93179,24907.0,146692,39212.0,559072,149444.0,68730,18372.0,108360,4477.0
9,10,8266.0,13922.0,8505.0,13835.0,5330.0,5460.0,0.930000,1001.000000,5.333000e+06,...,489556,91849.0,646210,121240.0,2937338,551095.0,381855,71643.0,942247,4295.0


In [22]:
GenericWellDataPrepped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 89 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Well Index                               16707 non-null  int64         
 1   TrueVerticalDepth_FT                     16707 non-null  float64       
 2   MeasuredDepth_FT                         16707 non-null  float64       
 3   UpperPerforation_FT                      16707 non-null  float64       
 4   LowerPerforation_FT                      16707 non-null  float64       
 5   PerforationInterval_FT                   16707 non-null  float64       
 6   LateralLength_FT                         16707 non-null  float64       
 7   ProppantLoad_LBSPerGAL                   16707 non-null  float64       
 8   ProppantIntensity_LBSPerFT               16707 non-null  float64       
 9   TotalProppant_LBS                      

In [23]:
#Removes Columns from numerical fields in data fram
for col in GenericWellDataPrepped.columns:
    if GenericWellDataPrepped[col].dtype == 'object':  # Check if the column contains string values
        # Remove commas and convert to numeric, setting errors to 'coerce' to handle non-numeric strings
        GenericWellDataPrepped[col] = pd.to_numeric(GenericWellDataPrepped[col].str.replace(',', ''), errors='coerce')

In [24]:
#Convert float data type to numeric and remove comma separators of numbers if necessary
#for col in GenericWellDataPrepped.columns:
   # if GenericWellDataPrepped[col].dtype == 'float':
    #    GenericWellDataPrepped[col] = GenericWellDataPrepped[col].pd.Int64Dtype()

In [25]:
GenericWellDataPrepped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 89 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Well Index                               16707 non-null  int64         
 1   TrueVerticalDepth_FT                     16707 non-null  float64       
 2   MeasuredDepth_FT                         16707 non-null  float64       
 3   UpperPerforation_FT                      16707 non-null  float64       
 4   LowerPerforation_FT                      16707 non-null  float64       
 5   PerforationInterval_FT                   16707 non-null  float64       
 6   LateralLength_FT                         16707 non-null  float64       
 7   ProppantLoad_LBSPerGAL                   16707 non-null  float64       
 8   ProppantIntensity_LBSPerFT               16707 non-null  float64       
 9   TotalProppant_LBS                      

In [26]:
#Export prepped data frame for model creation
GenericWellDataPrepped.to_csv('GenericWellDataPrepped.csv', index=False)